# Egocentric Video Mapper

Before starting make sure you have an available GPU. If you are unsure about whether you have an available GPU or if you want to check which GPU you will be working with run the next executble cell.

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-72b7ddec-7074-1583-0076-9281c065ae9d)


Now run the next cell to import the modules and functions that will be used for this AlphaLab

In [2]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 
from utils import VideoHandler, write_action_timestamp_csv
from optic_flow import OpticFlowCalculatorLK
from sync_videos import OffsetCalculator
from gaze_mapper import RulesBasedGazeMapper
from video_renderer import save_comparison_video, save_gaze_video

Fill out the path to the directory of the uncompressed Scene Video + Timeseries download from Cloud.

In [3]:
neon_timeseries_path='Path/To/NeonTimeSeriesFolder'
neon_vid_path= Path(neon_timeseries_path).rglob('*.mp4').__next__()

Fill out the path of the corresponding alternative egocentric view recording (please make sure that both videos have the same orientation).

In [4]:
action_vid_path='Path/To/AlternativeVideo'

Define an output directory where the different output files will be saved.

In [6]:
output_dir='Path/To/OutputFolder'

## Step 1: Video Synchronization

In this first step, optic flow in both videos will be calculated. Each optic flow result is saved to a csv file in an automatically created optic_flow subdirectory in the output directory. The csv file will contain the following columns:
- start: the start frame timestamp of the optic flow calculation
- end: the end frame timestamp of the optic flow calculation
- dx: Average horizontal displacement  
- dy: Average vertical displacement 
- angle: Average angular change in degrees

Optic flow is then used to obtain the time offset of the Action video with respect to the Neon Scene video. 



In [11]:
optic_flow_output_dir = Path(output_dir, 'optic_flow')
optic_flow_output_dir.mkdir(parents=True, exist_ok=True)
neon_of = OpticFlowCalculatorLK(video_dir=neon_vid_path)
neon_result = neon_of.process_video(output_file=Path(optic_flow_output_dir, 'neon_lk_of.csv'))

action_of = OpticFlowCalculatorLK(video_dir=action_vid_path)
action_result = action_of.process_video(output_file=Path(optic_flow_output_dir, 'action_lk_of.csv'))

File already exists, appending to it
File already exists, appending to it


Once the time offset is calculated, the action_worldtimestamps.csv can be generated with a similar format as the Neon world_timestamps.csv.

In [8]:
offset_calc=OffsetCalculator(src=action_result['dy'].values,src_timestamps=action_result['start'].values, dst=neon_result['dy'].values, dst_timestamps=neon_result['start'].values,resampling_frequency=500)

t_offset, pearson_corr = offset_calc.estimate_time_offset()
print(f'Estimated offset: {t_offset} seconds (Pearson correlation: {pearson_corr})')

write_action_timestamp_csv(neon_timeseries_path, VideoHandler(action_vid_path).timestamps+t_offset, saving_path=output_dir)

Last timestamp of Action camera recording (2024-05-23 14:48:20.852368384) is after the last timestamp of Neon Scene recording (2024-05-23 14:48:18.721666666)


Estimated offset: 2.016 seconds (Pearson correlation: 0.9192123717590794)


## Step 2: Map gaze data to the action video

After synchronizing both videos, it is time to obtain Neon gaze signal in the coordinate system of the other video. Here we make use of deep learning methods to guide the gaze transformation, for this notebook we will be using an implementation of Efficient LOFTR, if you are interested in using other algorithms we have implemented or in implementing one yourself check feature_matcher.py.

In [ ]:
image_matcher_choice='Efficient_LOFTR' # Options: 'Efficient_LOFTR', 'LOFTR', 'DISK_LightGlue', 'DeDoDe_LightGlue'


image_matcher_parameters = {'Efficient_LOFTR': {'model_type': 'opt', 'gpu_num': 0},'LOFTR':{"location": "indoor", "gpu_num": 0},'DISK_LightGlue': {"num_features": 2048, "gpu_num": 0},'DeDoDe_LightGlue': {"num_features": 5000, "gpu_num": 0}}
optic_flow_output_dir = Path(output_dir, 'optic_flow')
mapper_kwargs = {'neon_gaze_csv': Path(neon_timeseries_path, 'gaze.csv'),
                'neon_video_dir': neon_vid_path,
                'action_video_dir': action_vid_path,
                'neon_timestamps':  Path(neon_timeseries_path, 'world_timestamps.csv'),
                'action_timestamps': Path(output_dir, 'action_camera_timestamps.csv'),
                'neon_opticflow_csv': Path(optic_flow_output_dir, 'neon_lk_of.csv'),
                'action_opticflow_csv': Path(optic_flow_output_dir, 'action_lk_of.csv'),
                'image_matcher': image_matcher_choice,
                'image_matcher_parameters': image_matcher_parameters[image_matcher_choice],
                'patch_size': 1000}

mapper = RulesBasedGazeMapper(**mapper_kwargs)

Before running the mapping pipeline you can adjust how often (in seconds) you want the image correspondances refreshed. 

- The higher you set the time threshold the less time the mapping will take however it won't be as accurate as it could be. 

- The lower the time threshold the more time the mapping will take. 

If you leave the value at 'None' then for every gaze new imge correspondences will be obtained. We recommend using values smaller than 1 second.

In [ ]:
time_threshold = None 

In [ ]:
action_gaze_output_dir = Path(output_dir, f'mapped_gaze/{image_matcher_choice.lower()}')
action_gaze_output_dir.mkdir(parents=True, exist_ok=True)
mapping_kwargs = {'saving_path':Path(action_gaze_output_dir,f"action_gaze_lk.csv"),'refresh_time_thrshld':time_threshold}
mapper.map_gaze(**mapping_kwargs)

## Step 3 (OPTIONAL): Create visualization videos

For visualization purposes you can get a side-to-side rendering with gaze of the Neon Scene video and the alternative egocentric video. 


In [ ]:
action_gaze_dict = {image_matcher_choice.upper(): Path(action_gaze_output_dir,f"action_gaze_lk.csv")}
rendered_video_path = Path(output_dir, f"rendered_videos/neon_comparison_{image_matcher_choice.lower()}_lk.avi")
Path(rendered_video_path).parent.mkdir(parents=True, exist_ok=True)

comparison_video_args = {'action_video_path':action_vid_path,
                        'action_worldtimestamps_path':Path(output_dir, 'action_camera_timestamps.csv'),
                        'action_gaze_paths_dict':action_gaze_dict,
                        'neon_video_path':neon_vid_path,
                        'neon_worldtimestamps_path':neon_timestamps,
                        'neon_gaze_path':Path(neon_timeseries_path, 'gaze.csv'),
                        'save_video_path':rendered_video_path}

save_comparison_video(**comparison_video_args)  

You also can get only the alternative egocentric video  (at its original frame rate) with the overlaid gaze circle.

In [ ]:
gaze_video_args={'video_path':action_vid_path,
                'timestamps_path':Path(output_dir, 'action_camera_timestamps.csv'),
                'gaze_path':Path(action_gaze_output_dir, 'action_gaze_lk.csv'),
                'save_video_path':Path(output_dir, f"rendered_videos/{image_matcher_choice}_lk.avi")}
save_gaze_video(**gaze_video_args)